Latest update: March 4, 2025. Update from Sayem's chages:

*   May 28, 2025: final changes from Sayems
*   March 25: added exception file, fixed aliases, stress tested
*   March 4: Documentation updates
*   Feb 28: fixed the deep-dive




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
import requests
from urllib.parse import urlencode, quote_plus
import numpy as np
import sys
import os
import pandas as pd
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **Citing this code**
This code is the second version of a Expertise finding tool developed by Volz et al. 2023 (https://ui.adsabs.harvard.edu/abs/2023AAS...24210207V/abstract).<br>
It utilizes NASA ADS API to query for articles (refereed or not) in the "Astronomy" database (cite ADS).
Please, cite "Voltz et al. 2024" (https://ui.adsabs.harvard.edu/abs/2024RNAAS...8...69V/abstract) and "Kamal et al. 2025 (in prep) and refer to the README file in the github.

**Directory set up**<br>
The file *stopwords.txt* is utilized to create meaningful N-grams. Make sure to provide an accurate path in the following cell.<br> Also, the path will be used by the code in other instances to identify other useful files.

In [1]:
path_stop= '/Your Path/here/'
stop_file='stopwords.txt'
stop_dir=path_stop+stop_file
sys.path.append(path_stop)



NameError: name 'sys' is not defined

### You will need an API token from ADS

In [5]:
token = 'Your own token from ADS API page ' #Insert your ADS API token


In [6]:
!pip install ads

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00


### Useful libraries
In Version 3 we implemented an optional AI tool from Groq (https://github.com/groq/groq-python).
We also utlized rapidfuzz library to help with some string matching.


In [2]:
!pip install groq

Defaulting to user installation because normal site-packages is not writeable
Could not fetch URL https://pypi.org/simple/groq/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/groq/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)'))) - skipping
ERROR: Could not find a version that satisfies the requirement groq (from versions: none)
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)'))) - skipping
ERROR: No matching distribution found for groq


In [8]:

!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.9 MB/s eta 0:00:00


In [3]:
#For the TextAnalysis (TA) File, please refer to M. Volze et al. 2023 and other references above

import TextAnalysis as TA
import ADSsearcherpkg as AP

ModuleNotFoundError: No module named 'groq'

# **Example 1: Searching expertises of a single person based on their name**

#### The following examples will work with the **ads_search** function. These are the parameters of ads_search:

**name:** The name of the desired author (formatted "Last,First")  
**institution:** The name of the desired institution   (check also the ADS list of unique identifiers https://github.com/adsabs/CanonicalAffiliations/blob/master/parent_child.tsv)             

**year:** The year or range of years (formatted '[YEAR1 TO YEAR2]') of the desired publications  
**refereed:** Parameter that includes only refereed paper adding (formatted "property:refereed")  
**token:** API key for ADS  
**stop_dir:** Directory for list of stopwords for n-grams analysis  

**NEW in V3**

**second_auth:** Boolean parameter that determines whether second author publications are included (default=False)                                        
**groq_analysis:** Boolean parameter that determines whether groq expertise analysis will be run (default=False). Groq will try to best match the results of the N-grams to the list of AAS journal keywords.                                              
**deep_dive:** This will work in the file search of universities lists or singla universities (example 5 and 9 below). It will indentify all the individuals that published at that institutions and will re-run the search for all papers published by those researchers in a larger time-span to extract the publication history of these researchers. 

In [14]:
datf=AP.ads_search(name="Cucchiara, Antonino",
               token=token, refereed="property:refereed", stop_dir=stop_dir,groq_analysis=False)

I will search for papers matching the following criteria:
author:"^Cucchiara, Antonino"

I am now querying ADS.



In [15]:
datf
#datf.to_excel(path_stop+"output_Michail.xlsx")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Cucchiara, Antonino","None, None, None, None, None, None, None, None...","Cucchiara, Antonino, Cucchiara, A., Cucchiara,...","2016SSRv..202..143C, 2015ApJ...812..122C, 2015...","[GRBs as Probes of the IGM, Happy Birthday Swi...","2016-12-00, 2015-10-00, 2015-05-00, 2013-11-00...","[Gamma-ray bursts, Interstellar medium, Interg...","Space Telescope Science Institute, Baltimore, ...",Gamma-ray Bursts (GRBs) are the most powerful ...,"[2016arXiv160702747C, 10.48550/arXiv.1607.0274...","[(grb, 47), (redshift, 21), (grbs, 20), (host,...","[(gamma ray, 11), (ray burst, 9), (host galaxy...","[(gamma ray bursts, 6), (ray bursts grbs, 6), ...",Clean


In [17]:
import inspect
print(inspect.signature(AP.ads_search))

(name=None, institution=None, year=None, refereed='property:notrefereed OR property:refereed', token=None, stop_dir=None, second_auth=False, groq_analysis=True, deep_dive=False)


In [18]:
# To display the data frame run the following:
datf
print(len(datf.iloc[0]['Title']))
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

10


### **Example 1a: Single Name Search, including second author publications (groq analysis True)**

In [19]:
datf2=AP.ads_search(name="Cucchiara, Antonino",
               token=token, stop_dir=stop_dir,second_auth=True)

I will search for papers matching the following criteria:
(author:"^Cucchiara, Antonino" OR pos(author:"Cucchiara, Antonino",2))

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 1 rows in parallel with max_workers=2...
Raw response for row 0: ["gamma ray burst:general", "gamma ray burst:individual"]
Groq analysis complete.


/content/drive/MyDrive/Colab Notebooks/Sayem/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [20]:
datf2

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Subtopics
0,"Cucchiara, Antonino","None, None, None, None, None, None, None, None...","Strausbaugh, R., Strausbaugh, R., Strausbaugh,...","2025GCN.38805....1S, 2024GCN.38558....1S, 2024...","[GRB 250103B: LCOGT Optical Upper Limit, GRB 2...","2025-01-00, 2024-12-00, 2024-12-00, 2024-11-00...","[1957, 1916, 1671, Astrophysics - Instrumentat...","-, -, -, -, -, -, -, -, -, NASA HQ, NASA Headq...",We observed the Swift GRB 250103B field (Eyles...,"[2025GCN.38805....1S, 2024GCN.38558....1S, 202...","[(gcn, 334), (al, 320), (grb, 318), (band, 155...","[(al gcn, 297), (performed series, 100), (seri...","[(performed series exposures, 97), (hours grb ...",Clean,"[gamma ray burst:general, gamma ray burst:indi..."


In [24]:
#Check the expertise keywords associated by groq
print(datf2['Subtopics'][0])



['gamma ray burst:general', 'gamma ray burst:individual']


### **Example 1b: Single Name with LLM subtopics analysis**


In [ ]:
datf=AP.ads_search(name="Cucchiara, Antonino", token=token, stop_dir=stop_dir,groq_analysis=True)

I will search for papers matching the following criteria:
author:"^Cucchiara, Antonino"

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 1 rows in parallel with max_workers=2...
Raw response for row 0: ["gamma ray bursts:general", "galaxies:star formation", "cosmology:dark energy"]
Groq analysis complete.


/content/drive/MyDrive/Colab Notebooks/Sayem/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [ ]:
print(datf['Subtopics'][0])

['gamma ray bursts:general', 'galaxies:star formation', 'cosmology:dark energy']


# **Example 2: Searching expertises of ALL scientists that published as first authors when affiliated to single institution name**

The search will focus on papers and all authors that have published in the past 15 years at a specific institution (academic or otherwise): <br>
The format for a single institution is as follows: **institution="Institution Name"**. <br>
**Caveat**: It is possible that the institutions as input by the user does not match what has been cataloged in ADS, therefore if the final output is empty, make sure to try different versions of the institution names (e.g. Cal Poly Pomona, Cal Poly, California Polytechnic State University) to get the most complete list of authors.

The code will check the "exception.xlsx" file for similarities, but is always best to check affiliations as already exists in ADS (especially for foreign institutions

**CHECK** the list of institutions as [provided by ADS](https://https://github.com/adsabs/CanonicalAffiliations/blob/master/parent_child.tsv#L641)

In [25]:
datf=AP.ads_search(institution="Space Telescope Science Institute",refereed="property:refereed",
               token=token, year='[2016 TO 2017]', stop_dir=stop_dir, groq_analysis=False, deep_dive=True)

I will search for papers matching the following criteria:
pos(institution:"Space Telescope Science Institute",1) AND pubdate:[2016 TO 2017]

I am now querying ADS.

Unique authors from institution search: ['Boyer, M. L.', 'Brown, T. M.', 'Gennaro, Mario', 'Temim, Tea', 'Sohn, Sangmo Tony', 'Zasowski, G.', 'LaMassa, Stephanie M.', 'Bond, Howard E.', 'Peña-Guerrero, María A.', 'Gilbert, Karoline', 'Jones, O. C.', 'Pirzkal, Norbert', 'Ryan, Russell E., Jr.', 'Goudfrooij, Paul', 'Boyce, H.', 'Anderson, J.', 'Dixon, William V.', 'Bellini, A.', 'Brown, Robert A.', 'Tumlinson, Jason', 'Lubow, Stephen H.', 'Walborn, Nolan R.', 'Gordon, K. D.', 'Kains, N.', 'Williams, Brian J.', 'Roman-Duval, Julia', 'Ryon, J. E.', 'Snyder, Gregory F.', 'Bohlin, Ralph C.', 'Jones, Olivia C.', 'Correnti, Matteo', 'Fischer, William J.', 'Chastenet, Jérémy', 'Johnson, Christopher B.', 'Strolger, Louis-Gregory', 'Sparks, W. B.', 'Watkins, Laura L.', 'Chiaberge, M.', 'Gompertz, B.', 'LaMassa, S. M.', 'Casertano, Ste

In [26]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Anderson, J.","None, None, None, None, None, None, None, None...","Anderson, Jay, Anderson, Jay, Anderson, Jay, A...","2017JRASC.111..247A, 2017JRASC.111..191A, 2017...","[News Notes / En manchettes, News Notes / En m...","2017-12-00, 2017-10-00, 2017-09-00, 2017-09-00...","[Space Telescope Science Institute, STScI, Jam...","-, -, Space Telescope Science Institute, Space...",", , This report documents the verification by ...","[2017JRASC.111..247A, 2017JRASC.111..191A, 201...","[(ii, 30), (dark, 18), (psf, 16), (model, 13),...","[(ii region, 8), (long dark, 7), (sn ii, 7), (...","[(host ii region, 5), (type ii supernova, 4), ...",Dirty
1,"Baldwin, A. T.","None, None","Baldwin, A. T., Baldwin, A. T.","2016yCat..18270012B, 2016ApJ...827...12B",[VizieR Online Data Catalog: HSTPROMO catalogs...,"2016-11-00, 2016-08-00","[Clusters: globular, Velocity dispersion, Star...","-, Department of Physics &amp; Astronomy, Loui...","Although Paper I (Bellini+, 2014, J/ApJ/797/11...","[10.26093/cds/vizier.18270012, 2016yCat..18270...","[(cluster, 8), (mass, 8), (gc, 5), (star, 5), ...","[(velocity dispersion, 4), (average mass, 4), ...","[(proper motion catalogs, 2), (galactic globul...",Clean
2,"Banzatti, A.","None, None, None, None, None","Banzatti, A., Banzatti, A., Banzatti, A., Banz...","2017abnw.conf.2016B, 2017LPICo2043.2016B, 2017...",[Revealing the physical and thermo-chemical ev...,"2017-08-00, 2017-08-00, 2017-01-00, 2016-06-00...","[circumstellar matter, molecular processes, pl...","University of Arizona, -, Space Telescope Scie...",We report results of a large campaign to revea...,"[2017abnw.conf.2016B, 2017LPICo2043.2016B, arX...","[(disk, 23), (gas, 11), (gap, 11), (planet, 10...","[(snow line, 6), (planet forming, 5), (gas emi...","[(report results large, 2), (results large cam...",Dirty
3,"Bellini, A.","None, None, None, None, None, None, None, None...","Bellini, A., Bellini, A., Bellini, A., Bellini...","2017acs..rept...12B, 2017romanrept.1702B, 2017...","[The Hubble Space Telescope ""Program of Last R...","2017-09-00, 2017-09-00, 2017-08-00, 2017-08-00...","[Hubble Space Telescope, HST, Advanced Camera ...","Space Telescope Science Institute, Space Teles...","Every year, the Space Telescope Science Instit...","[2017acs..rept...12B, 2017romanrept.1702B, arX...","[(hubble, 14), (cluster, 14), (space, 13), (te...","[(space telescope, 12), (proper motion, 9), (h...","[(hubble space telescope, 7), (space telescope...",Dirty
4,"Blevins, Sandra M.",None,"Blevins, Sandra M.",2016ApJ...818...22B,[Measurements of Water Surface Snow Lines in C...,2016-02-00,"[astrochemistry, molecular processes, protopla...","Space Telescope Science Institute, Baltimore, ...",We present deep Herschel-PACS spectroscopy of ...,"[10.3847/0004-637X/818/1/22, arXiv:1512.07197,...","[(water, 11), (gas, 9), (dust, 8), (surface, 7...","[(gas dust, 7), (water abundance, 6), (surface...","[(surface water abundance, 3), (gas dust tempe...",Clean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,"Whitmore, Bradley C.","None, None, None, None, None","Whitmore, B., Whitmore, Bradley C., Whitmore, ...","2017ASPC..512...77W, 2017IAUS..316..202W, 2016...","[Version 1 of the Hubble Source Catalog, Unive...","2017-12-00, 2017-03-00, 2016-10-00, 2016-06-00...","[Surveys, Photometry: HST, astrometry, catalog...","STScI, Baltimore, MD, 21218, USA, Space Telesc...",The Hubble Source Catalog (HSC) is designed to...,"[2017adass..25...77W, 2018ASPC..512...77W, 201...","[(source, 24), (catalog, 16), (hubble, 11), (w...","[(source list, 9), (hubble source, 5), (source...","[(hubble source catalog, 5), (hubble space tel...",Dirty
93,"Williams, Brian J.","None, None, None, None, None, None, None, None...","Williams, Brian J., Williams, Brian, Williams,...","2017HEAD...1630504W, 2017hst..prop15120

# **Example 3: Searching a single author publication while affiliated to a specific institution**

The search will focus on papers published by a single author while they are affiliated to a specific institution, in the past 15 years:<br>

The format for a single author and institution is as follows: **name= 'Last, First', institution= 'Institution Name'**.

In [27]:
datf=AP.ads_search(name= 'Cucchiara, Antonino', institution="University of California, Berkeley",
               token=token,stop_dir=stop_dir, year='[2010 TO 2014]')

I will search for papers matching the following criteria:
author:"^Cucchiara, Antonino" AND pos(institution:"University of California, Berkeley",1) AND pubdate:[2010 TO 2014]

I am now querying ADS.

DataFrame is empty! Trying alternative search using affiliation instead of institution.
Alternative query:
author:"^Cucchiara, Antonino" AND pos(aff:"University of California, Berkeley",1) AND pubdate:[2010 TO 2014]

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 1 rows in parallel with max_workers=2...
Raw response for row 0: ["cosmology:dark energy", "gamma rays:galaxies", "stars:gamma ray burst:general"]
Groq analysis complete.


/content/drive/MyDrive/Colab Notebooks/Sayem/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [28]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Subtopics
0,"Cucchiara, Antonino","University of California, Berkeley, University...","Cucchiara, A., Cucchiara, Antonino, Cucchiara, A.","2011ApJ...743..154C, 2011noao.prop..484C, 2011...",[Constraining Gamma-Ray Burst Emission Physics...,"2011-12-00, 2011-08-00, 2011-07-00",[gamma-ray burst: individual: GRB 110205A GRB ...,"Department of Astronomy, University of Califor...",Understanding the origin and diversity of emis...,"[2011arXiv1107.3352C, arXiv:1107.3352, 10.1088...","[(grb, 11), (redshift, 11), (burst, 7), (grbs,...","[(optical infrared, 4), (redshift burst, 3), (...","[(gamma ray bursts, 2), (ray bursts grbs, 2), ...",Clean,"[cosmology:dark energy, gamma rays:galaxies, s..."


# **Example 4: Searching a single author name within a different time-frame**

The search will focus on papers from one single author that were published in a different time-frame. There are two options for doing so:
   - A single year (e.g. 2010): in this case the code will query ADS for articles published by the specified authors between one year prior to 4  years after. So searching year='2010' will search articles between 2009 and 2014<br>
   - A year range: in this case the syntax is year='[YEAR TO YEAR]' (e.g. year='[2009 TO 2023]') <br>

The format for a single author name remains the same as before: **name= 'Last, First'**. <br>

Here are two examples:
- Searching for Dr. Pepper's articles between year 1999 and 2004
- Searching for Dr. Pepper's articles between year 2019 and 2023

In [ ]:
datf=AP.ads_search(name= 'Pepper, Joshua', year='2000',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
author:"^Pepper, Joshua", pubdate:[1999 TO 2004]

I am now querying ADS.



In [ ]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Pepper, Joshua","None, None, None, None, None","Pepper, J., Pepper, Joshua, Pepper, J., Pepper...","2004AAS...20517102P, 2004AIPC..713..185P, 2003...","[Deployment and Initial Data from KELT, KELT: ...","2004-12-00, 2004-06-00, 2003-09-00, 2002-12-00...","[97.82.Fs, 97.82.Cp, Substellar companions, pl...","OSU, Department of Astronomy, 4055 McPherson L...",The Kilodegree Extremely Little Telescope (KEL...,"[2004AAS...20517102P, 2005BAAS...37..375P, 200...","[(star, 15), (transit, 13), (telescope, 12), (...","[((bright, star), 9), ((transit, bright), 8), ...","[((transits, bright, stars), 8), ((bright, sta...","Dirty, Dirty, Dirty, Dirty, Dirty"


In [ ]:
datf=AP.ads_search(name= 'Pepper, Joshua', year='[2019 TO 2023]',
               token=token, stop_dir=stop_dir)

In [ ]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

# **Example 5: Searching a single institution name within a specific time-frame**

The search will focus on authors that publishes as first authors affiliated to a specific institution in a defined timespan. <br>
The format for a author name is the same in previous example (**"Last name, First name"**) and specified year range is similar to the option provided earlier:<br>
   - A single year (e.g. 2010): in this case the code will query ADS for articles published by the specified authors between one year prior to 4  years after. So searching year='2010' will search articles between 2009 and 2014<br>
   - A year range: in this case the syntax is year='[YEAR TO YEAR]' (e.g. year='[2009 TO 2023]') <br>

Following we present two examples:

In [29]:
datf=AP.ads_search(institution="University of Southern Mississippi",year='2000',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
pos(institution:"University of Southern Mississippi",1) AND pubdate:[1999 TO 2004]

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 1 rows in parallel with max_workers=2...
Raw response for row 0: ["astronomical instrumentation methods and techniques:instrumentation: miscellaneous", "planetary systems:Mars"]
Groq analysis complete.


/content/drive/MyDrive/Colab Notebooks/Sayem/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [30]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Subtopics
0,"Buchanan, Randy K.",University of Southern Mississippi,"Buchanan, Randy K.",2003AIPC..684.1121B,[Simulating Martian Temperatures],2003-09-00,"[07.20.Mc, 95.55.-n, Cryogenics, refrigerators...","Electronic Engineering Technology, University ...",The Mars Electrostatic Chamber (MEC) was desig...,"[2003AIPC..684.1121B, 10.1063/1.1627280]","[(temperature, 9), (control, 5), (mar, 4), (ch...","[(temperature control, 3), (pressure atmospher...","[(mars electrostatic chamber, 1), (electrostat...",Dirty,[astronomical instrumentation methods and tech...


In [31]:
datf=AP.ads_search(institution= "University of Southern Mississippi", year='[2019 TO 2023]',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
pos(institution:"University of Southern Mississippi",1) AND pubdate:[2019 TO 2023]

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 9 rows in parallel with max_workers=2...
Raw response for row 1: ["planetary systems:general", "planetary systems: Moon"]
Raw response for row 0: ["physical data and processes:gravitational waves", "astronomical instrumentation methods and techniques:instrumentation: detectors", "astronomical instrumentation methods and techniques:methods: data analysis"]
Raw response for row 2: I'm ready to analyze the researcher's n-grams. Please provide the lists of n-grams (top words, bigrams, and trigrams) extracted from the researcher's ADS abstracts.
Raw response for row 3: ["physical data and processes: nucleosynthesis, abundances", "stars: abundances"]
Raw response for row 4: ["physical data and processes:geochemistry", "physical data and processes:astroche

/content/drive/MyDrive/Colab Notebooks/Sayem/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [32]:
datf

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Subtopics
0,"Brockamp, Betina",University of Southern Mississippi,"Brockamp, Betina",2023nsbp.confE..26B,[Exploring near-inertial oscillations and dyna...,2023-11-00,[],University of Southern Mississippi,This project investigates the dynamics and cha...,[2023nsbp.confE..26B],"[(inertial, 4), (study, 3), (oscillation, 3), ...","[(inertial oscillation, 3), (offshore mobile, ...","[(offshore mobile bay, 2), (project investigat...",Dirty,[physical data and processes:gravitational wav...
1,"Capper, Daniel","University of Southern Mississippi, University...","Capper, Daniel, Capper, Daniel","2022SpPol..5901462C, 2019SpPol..4901325C",[What Should We Do with Our Moon?: Ethics and ...,"2022-02-00, 2019-08-00","[Environmental justice, Environmental preserva...","School of Humanities, University of Southern M...",The planned expansion of the human presence on...,"[10.1016/j.spacepol.2021.101462, 2022SpPol..59...","[(environmental, 8), (human, 7), (moon, 7), (r...","[(baseline ecology, 3), (environmental science...","[(baseline ecology method, 2), (planned expans...",Dirty,"[planetary systems:general, planetary systems:..."
2,"Hayes, C. T.",University of Southern Mississippi,"Hayes, C. T.",2020AGUFMPP011..07H,[The Composition and Flux of Seafloor Sediment...,2020-12-00,"[1051 Sedimentary geochemistry, GEOCHEMISTRY, ...","University of Southern Mississippi, Stennis Sp...",,[2020AGUFMPP011..07H],[],[],[],Dirty,[]
3,"Redmond, N.",University of Southern Mississippi,"Redmond, N.",2020AGUFMPP011..06R,[Potential Remobilization and Sources of <SUP>...,2020-12-00,"[1051 Sedimentary geochemistry, GEOCHEMISTRY, ...","School of Ocean Science and Engineering, Unive...",Due to its sensitivity to changes in redox con...,[2020AGUFMPP011..06R],"[(authigenic, 4), (record, 4), (remobilization...","[(authigenic uranium, 2), (potential complicat...","[(due sensitivity redox, 1), (sensitivity redo...",Dirty,"[physical data and processes: nucleosynthesis,..."
4,"Redmond, N. A.",University of Southern Mississippi,"Redmond, N. A.",2023GeCoA.363...40R,[Anomalous <SUP>234</SUP>U/<SUP>238</SUP>U iso...,2023-12-00,"[Trace elements, Deep sea diagenesis, Sediment...","University of Southern Mississippi, School of ...",Uranium is a redox-sensitive trace element tha...,"[2023GeCoA.363...40R, 10.1016/j.gca.2023.09.015]","[(sediment, 6), (anomalous, 4), (redox, 3), (s...","[(pelagic sediment, 2), (pore water, 2), (uran...","[(uranium redox sensitive, 1), (redox sensitiv...",Dirty,"[physical data and processes:geochemistry, phy..."
5,"Ringermacher, H. I.",University of Southern Mississippi,"Ringermacher, H. I.",2020MNRAS.494.2158R,[Reaffirmation of cosmological oscillations in...,2020-05-00,"[cosmological parameters, dark matter, Astroph...","Department of Physics and Astronomy, Universit...",We observe damped temporal oscillations in the...,"[10.48550/arXiv.1901.10311, arXiv:1901.10311, ...","[(oscillation, 7), (dark, 4), (matter, 4), (te...","[(dark matter, 4), (temporal oscillation, 3), ...","[(temporal oscillations scale, 2), (oscillatio...",Clean,"[cosmology:dark energy, cosmology:dark matter]"
6,"Ringermacher, Harry I.",University of Southern Mississippi,"Ringermacher, Harry I.",2019AAS...23331501R,[Confirmation of Inflaton-Like Oscillations in...,2019-01-00,[],"Physics and Astronomy, U.of Southern Mississip...",We have confirmed the presence of low frequenc...,[2019AAS...23331501R],"[(model, 5), (oscillation, 4), (matter, 4), (t...","[(dark matter, 3), (pantheon compilation, 2), ...","[(confirmed presence low, 1), (presence low fr...",Clean,"[cosmology:dark energy, cosmology:cosmological..."
7,"Sirola, Christopher",University of Southern Mississippi,"Sirola, Christopher",2021PhTea..59..590S,[Lunar astrometry],2021-10-00,[],"Department of Physics &amp; Astronomy, Univers...",A question for science t

# **Example 6: Searching a single Author, at a specific institution and within a specific time-frame**

The following example combines several of the previous ones in a single search.
Specifically:<br>
   - A single author<br>
   - Affiliated to a single institutions<br>
   - In a specific time frame of publications<br>
    
Please, refer to the previous examples for the sintax required. <br>
Here is an example

In [33]:
datf=AP.ads_search(name= 'Brown, Beth A.', institution="Howard university",year='[2009 TO 2022]',
               token=token, stop_dir=stop_dir)

I will search for papers matching the following criteria:
author:"^Brown, Beth A." AND pos(institution:"Howard university",1) AND pubdate:[2009 TO 2022]

I am now querying ADS.

Running Groq subtopics analysis on ADS results...
Extracting subtopics for 1 rows in parallel with max_workers=2...
Raw response for row 0: ["physical data and processes:gravitational physics", "physical data and processes:black hole physics"]
Groq analysis complete.


/content/drive/MyDrive/Colab Notebooks/Sayem/LlamaModelV2.py:525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Subtopics"] = subtopics_column


In [34]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,Subtopics
0,"Brown, Beth A.",Howard university,"Brown, Beth A.",2010AIPC.1280....3B,[Causal Structures of Dynamic Black Holes],2010-10-00,"[black holes, space-time configurations, satel...","NASA Goddard Space Flight Center, Greenbelt, M...","Dynamic space-times, especially those manifest...","[10.1063/1.3507200, arXiv:0904.4192, 2010AIPC....","[(black, 3), (hole, 3), (dynamic, 2), (behavio...","[(black hole, 3), (hole presented, 2), (dynami...","[(black hole presented, 2), (dynamic space tim...",Dirty,[physical data and processes:gravitational phy...


# **Example 7: Searching through a list of institutions**

The search will focus on papers from a list of institutions, so the input is a csv file that has multiple institution names stored in it. This will then find all papers from those institutions (**see CAVEATS in Example 2 above related to Institution Names)**:<br>

The input file has to be a .csv file (e.g."top10inst.csv"), and must contain at least one column titled  **"Current Institution"** or **"Institution"** (the first cell of the column is usually interpreted as such). The file can contains other columns, they will be ignored.<br>
If the file is in a different directory than the one where the code it, include the whole path. <br>

The code will run as in Example 2 above for each institutions and append the results at each iteration providing a final dataframe with all the researchers at all the institutions in the list provided.<br>

**Usage:** the code will read the file and ask several questions, including which column should be used, which type of search needs to be executed (institutions list or name list), what year range, deep dive and groq analysis.

In [35]:
datf=AP.run_file_search(filename='/content/drive/MyDrive/Colab Notebooks/Sayem/Institutions_Example.csv',
                          token=token, stop_dir=stop_dir)



What type of search do you want to conduct?
-Enter 'name' for Name Search - search by author name
-Enter 'institution' for Institution Search - search by institution

Enter search type: ('name' or 'institution'):
institution
You are running 'institution' search.

Listed are the available columns from your dataset: Institution
Enter the name of the column that contains the data for 'institution' search: Institution
Do you want to run a deep dive search (re-run for each author) for institution search? (y/n) [n]: n
Enter the year range for your search (format: [YYYY TO YYYY] or a 4-digit year, default: [2003 TO 2030]): 
Do you want refereed papers only? (y/n) [y]: y
Do you want to run Groq subtopics analysis on the ADS results? (y/n) [n]: n
Searching for results...
Processing institution: University of California, Berkeley
I will search for papers matching the following criteria:
pos(institution:"University of California, Berkeley",1) AND pubdate:[2003 TO 2030]

I am now querying ADS.

D

KeyboardInterrupt: 

In [ ]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")
datf.to_excel(path_stop+"Expertise_Carn25_R1.xlsx")

# **Example 8: Searching through a list of Authors names**

The search will focus on papers from a list of authors names (similar format as Example 1 above, **'Last, First'**). <br>
The input is a .csv file that has multiple authors names stored in it under a column Title: **"Name"**. <br>
The ADS search will focus on the period 2003 to 2023.
<br>
If the file is in a different directory than the one where the code it, include the whole path. <br>

The code will then execute the search one name after the other and uppend each result to the previous one.<br>
In the following example we use, for convenience, the same example file as before which also contain a list of researchers names.


In [36]:
datf=AP.run_file_search(filename='/content/drive/MyDrive/Colab Notebooks/Sayem/Names_Example.csv',
               token=token, stop_dir=stop_dir)



What type of search do you want to conduct?
-Enter 'name' for Name Search - search by author name
-Enter 'institution' for Institution Search - search by institution

Enter search type: ('name' or 'institution'):
name
You are running 'name' search.

Listed are the available columns from your dataset: Name
Enter the name of the column that contains the data for 'name' search: Name
Do you want to include search by second author? (y/n) [n]: y
Enter the year range for your search (format: [YYYY TO YYYY] or a 4-digit year, default: [2003 TO 2030]): [2022 TO 2024]
Do you want refereed papers only? (y/n) [y]: y
Do you want to run Groq subtopics analysis on the ADS results? (y/n) [n]: n
Searching for results...
I will search for papers matching the following criteria:
(author:"^Browning, Matthew" OR pos(author:"Browning, Matthew",2)) AND pubdate:[2022 TO 2024]

I am now querying ADS.

Completed 1 searches - Processed name: Browning, Matthew (including second author)
I will search for papers m

In [37]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"RTF25_expertise.csv")
#datf.to_excel(path_stop+"Expertise_Labastro.xlsx")

,Input Author,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
0,"Browning, Matthew",None,"Käpylä, Petri J.",2023SSRv..219...58K,[Simulations of Solar and Stellar Dynamos and ...,2023-10-00,"[Dynamo, Magnetohydrodynamics, Simulation, Tur...","Institute for Astrophysics and Geophysics, Uni...",We review the state of the art of three dimens...,"[2023arXiv230516790K, 10.1007/s11214-023-01005...","[(simulation, 5), (dynamo, 3), (numerical, 2),...","[(simulation solar, 2), (review state, 1), (st...","[(review state art, 1), (state art dimensional...",Dirty
1,"Cruz, Kelle",None,None,None,"[N, o, n, e]",None,"[N, o, n, e]",None,None,"[N, o, n, e]",[],[],[],Dirty
2,"Gawiser, Eric","None, None, None, None, None, None","Firestone, Nicole M., Moskowitz, Irene, Lee, K...","2024ApJ...974..217F, 2024ApJ...967L...6M, 2024...",[ODIN: Improved Narrowband Lyα Emitter Selecti...,"2024-10-00, 2024-05-00, 2024-02-00, 2023-06-00...","[Emission line galaxies, Cosmology, Galaxy evo...","Department of Physics and Astronomy, Rutgers, ...",Lyman-alpha-emitting galaxies (LAEs) are typic...,"[10.48550/arXiv.2312.16075, 2023arXiv231216075...","[(galaxy, 33), (sample, 22), (redshift, 15), (...","[(training sample, 11), (star formation, 10), ...","[(representative training sample, 3), (legacy ...",Clean


In [ ]:
print(len(datf.iloc[0]['Title']))

29


# Example 9: Searching a list of Authors at Institutionsd during specific times

In [ ]:
datf=AP.run_file_search(filename='RTF_2025.csv',
               token=token, stop_dir=stop_dir)

In [ ]:
# To display the data frame run the following:
datf
# To save it in a excel format run the following:
#datf.to_csv(path_stop+"output.csv")